In [69]:
import numpy as np
import random
from typing import List, Tuple, Iterable, Set
from itertools import combinations
from math import inf

In [70]:
Point = Tuple[int, int]

def generate_instance(num_rows_columns: int, D: int, J: int, seed: int = 42) -> Tuple[List[Point], List[Point]]:
    if D + J > num_rows_columns * num_rows_columns:
        raise ValueError("グリッド上の点数より D+J が大きいです。")
    rng = random.Random(seed)
    grid = [(x, y) for x in range(1, num_rows_columns + 1) for y in range(1, num_rows_columns + 1)]
    rng.shuffle(grid)
    demand_points = grid[:D]
    candidate_sites = grid[D:D+J]
    return demand_points, candidate_sites

def compute_distances(demand_points: List[Point], candidate_sites: List[Point]) -> np.ndarray:
    dp = np.array(demand_points, dtype=float)  # (D,2)
    cs = np.array(candidate_sites, dtype=float)  # (J,2)
    diff = dp[:, None, :] - cs[None, :, :]      # (D,J,2)
    return np.sqrt((diff ** 2).sum(axis=2))     # (D,J)

def compute_wij_matrix(distances: np.ndarray, alpha: float = 0.0, beta: float = 0.1) -> np.ndarray:
    d = np.asarray(distances, dtype=float)
    with np.errstate(over='ignore'):
        w = np.exp(-alpha * d) / (1.0 + beta * d)
    return w

def compute_Ui_from_set(wij: np.ndarray, J_set: Iterable[int]) -> np.ndarray:
    J_list = list(J_set)
    if len(J_list) == 0:
        return np.zeros(wij.shape[0])
    return wij[:, J_list].sum(axis=1)  # D 次元

# ====== Stackelberg評価：w を直接使用（基礎効用=0 とみなす） ======
def leader_objective_from_sets(w: np.ndarray, h: np.ndarray, x_set: Set[int], y_set: Set[int]) -> float:
    """
    A_i = sum_{j in x} w_ij, B_i = sum_{j in y} w_ij
    L^+(x,y) = sum_i h_i * A_i / (A_i + B_i)
    """
    A = compute_Ui_from_set(w, x_set)
    B = compute_Ui_from_set(w, y_set)
    denom = A + B
    share_L = np.where(denom > 0, A / denom, 0.0)
    return float((h * share_L).sum())

def competitor_objective_from_sets(w: np.ndarray, h: np.ndarray, x_set: Set[int], y_set: Set[int]) -> float:
    A = compute_Ui_from_set(w, x_set)
    B = compute_Ui_from_set(w, y_set)
    denom = A + B
    share_F = np.where(denom > 0, B / denom, 0.0)
    return float((h * share_F).sum())

# ====== フォロワー最善応答 ======
def best_response_follower_exact(w: np.ndarray, h: np.ndarray, x_set: Set[int], nJ: int, r: int) -> Tuple[Set[int], float]:
    best_y, best_val = set(), -inf
    available = [j for j in range(nJ) if j not in x_set]
    # 0..r すべて試す（小規模専用）
    for k in range(min(r, len(available)) + 1):
        for combo in combinations(available, k):
            y_set = set(combo)
            val = competitor_objective_from_sets(w, h, x_set, y_set)
            if val > best_val:
                best_val, best_y = val, y_set
    return best_y, best_val

def best_response_follower_greedy(w: np.ndarray, h: np.ndarray, x_set: Set[int], nJ: int, r: int) -> Tuple[Set[int], float]:
    """
    大規模用：貪欲に r 回だけ周辺利得が最大のサイトを追加
    """
    y_set: Set[int] = set()
    candidates = [j for j in range(nJ) if j not in x_set]
    # 事前計算（現在のA,Bと分母を持っておく）
    A = compute_Ui_from_set(w, x_set)
    B = np.zeros_like(A)

    for _ in range(min(r, len(candidates))):
        best_j, best_gain = None, -inf
        denom = A + B
        base_shareF = np.where(denom > 0, B / denom, 0.0)
        base_val = float((h * base_shareF).sum())
        # 候補ごとの周辺利得
        for j in candidates:
            if j in y_set:
                continue
            Bj = B + w[:, j]
            denom_j = A + Bj
            shareF_j = np.where(denom_j > 0, Bj / denom_j, 0.0)
            val_j = float((h * shareF_j).sum())
            gain = val_j - base_val
            if gain > best_gain:
                best_gain, best_j = gain, j
        if best_j is None:
            break
        y_set.add(best_j)
        B = B + w[:, best_j]
        candidates.remove(best_j)

    final_val = competitor_objective_from_sets(w, h, x_set, y_set)
    return y_set, final_val

"""
def best_response_of_competitor(w: np.ndarray, h: np.ndarray, x_set: Set[int], nJ: int, r: int) -> Tuple[Set[int], float]:
    # サイズで厳密/貪欲を出し分け
    if nJ <= 25:
        return best_response_follower_exact(w, h, x_set, nJ, r)
    return best_response_follower_greedy(w, h, x_set, nJ, r)
"""
def best_response_of_competitor(w: np.ndarray, h: np.ndarray, x_set: Set[int], nJ: int, r: int) -> Tuple[Set[int], float]:
    # サイズで厳密/貪欲を出し分け
    #if nJ <= 25:
    return best_response_follower_exact(w, h, x_set, nJ, r)
    #return best_response_follower_greedy(w, h, x_set, nJ, r)

# ====== リーダー側の探索 ======
def solve_leader(w: np.ndarray, h: np.ndarray, nJ: int, p: int, r: int):
    """
    小規模：厳密全列挙
    大規模：前進貪欲（1 施設ずつ追加し、その都度フォロワー最善応答を反映）
    """
    
    
    # if nJ <= 25:
    best_val, best_x, best_y = -inf, None, None
    for combo in combinations(range(nJ), p):
        x_set = set(combo)
        y_set, _ = best_response_of_competitor(w, h, x_set, nJ, r)
        val = leader_objective_from_sets(w, h, x_set, y_set)
        if val > best_val:
            best_val, best_x, best_y = val, x_set, y_set
    return best_x, best_y, best_val

    """
    # 大規模：貪欲
    x_set: Set[int] = set()
    remaining = set(range(nJ))
    for _ in range(p):
        best_j, best_val = None, -inf
        for j in list(remaining):
            trial_x = set(x_set)
            trial_x.add(j)
            y_set, _ = best_response_of_competitor(w, h, trial_x, nJ, r)
            val = leader_objective_from_sets(w, h, trial_x, y_set)
            if val > best_val:
                best_val, best_j = val, j
        if best_j is None:
            break
        x_set.add(best_j)
        remaining.remove(best_j)
    # 最終 y と値
    y_set, _ = best_response_of_competitor(w, h, x_set, nJ, r)
    final_val = leader_objective_from_sets(w, h, x_set, y_set)
    return x_set, y_set, final_val
    """


In [71]:
np.set_printoptions(edgeitems=3, threshold=50, linewidth=140, suppress=True)

D, J = 30, 30
num_rows_columns = 50
demand_points, candidate_sites = generate_instance(num_rows_columns, D, J, seed=None)

print("Demand Points:")
print(demand_points)

print("\nCandidate Sites:")
print(candidate_sites)

alpha = 0
beta = 0.1
p = 2
r = 2

# 均等需要
h_i = np.full(D, 1.0 / D)

# “既存”開設集合（必要ならインデックスを入れて固定寄与に使う） 
J_L_fixed: Set[int] = set() 
J_F_fixed: Set[int] = set()


Demand Points:
[(24, 41), (31, 3), (39, 26), (38, 10), (15, 6), (3, 35), (2, 32), (24, 42), (32, 18), (14, 11), (18, 12), (34, 42), (44, 18), (36, 38), (5, 34), (24, 26), (50, 18), (28, 32), (42, 36), (31, 43), (20, 28), (6, 28), (12, 11), (42, 33), (30, 38), (24, 39), (37, 27), (46, 26), (5, 6), (44, 1)]

Candidate Sites:
[(6, 26), (12, 17), (13, 3), (31, 41), (37, 39), (39, 23), (1, 47), (29, 3), (8, 3), (48, 11), (38, 37), (5, 31), (29, 25), (45, 22), (31, 32), (50, 33), (29, 27), (13, 19), (37, 13), (1, 27), (40, 22), (33, 26), (39, 27), (43, 1), (9, 38), (4, 28), (7, 13), (12, 22), (43, 18), (43, 3)]


In [72]:
import time

# ==== 計測開始 ====
t0 = time.perf_counter()

# 距離
distances = compute_distances(demand_points, candidate_sites)
t1 = time.perf_counter()
print(f"\n[time] compute_distances: {t1 - t0:.6f} s")

# w_ij
w = compute_wij_matrix(distances, alpha, beta)
t2 = time.perf_counter()
print(f"[time] compute_wij_matrix: {t2 - t1:.6f} s")

# U の参考出力
Ui_L = compute_Ui_from_set(w, J_L_fixed)
Ui_F = compute_Ui_from_set(w, J_F_fixed)
t3 = time.perf_counter()
print(f"[time] compute_Ui_from_set (L+F): {t3 - t2:.6f} s")

# Stackelberg 最適化
x_star, y_star, L_val = solve_leader(w, h_i, nJ=J, p=p, r=r)
t4 = time.perf_counter()
print(f"[time] solve_leader: {t4 - t3:.6f} s")

# 競合側の目的
comp_val = competitor_objective_from_sets(w, h_i, x_star, y_star)
t5 = time.perf_counter()
print(f"[time] competitor_objective_from_sets: {t5 - t4:.6f} s")

print(f"[time] TOTAL: {t5 - t0:.6f} s")

print("\nOptimal leader facility choices (x_j = 1):", sorted(x_star)) 
print("Optimal follower facility choices in response (y_j = 1):", sorted(y_star))

print(f"Leader's objective value L^+(x,y) = {L_val:.6f}")
print(f"Competitor's objective value = {comp_val:.6f}")


[time] compute_distances: 0.000117 s
[time] compute_wij_matrix: 0.000153 s
[time] compute_Ui_from_set (L+F): 0.000083 s
[time] solve_leader: 1.659296 s
[time] competitor_objective_from_sets: 0.000168 s
[time] TOTAL: 1.659818 s

Optimal leader facility choices (x_j = 1): [14, 16]
Optimal follower facility choices in response (y_j = 1): [12, 21]
Leader's objective value L^+(x,y) = 0.502996
Competitor's objective value = 0.497004
